In [ ]:
from hallucination_eval import HalluEvaluator
import json
from yfiles_jupyter_graphs import GraphWidget
import networkx as nx

# Create an instance of HalluEvaluator
graph_path1 = r"C:\Users\omar2\OneDrive\Desktop\graphdistance-main\graphdistance-main\dspy_dfkg\testcase\node_distance\08_a.json"
graph_path2 = r"C:\Users\omar2\OneDrive\Desktop\graphdistance-main\graphdistance-main\dspy_dfkg\testcase\node_distance\08_b.json"
hallu_evaluator = HalluEvaluator(graph_pathA=graph_path1, graph_pathB=graph_path2)

# Access graph1 and graphx1 from the instance
graph1 = hallu_evaluator.graph1
graphx1 = hallu_evaluator.graphx1

# Create new graph combining structure and attributes
combined_graph = nx.Graph()

# Copy structure from graphx1
combined_graph.add_nodes_from(graphx1.nodes())
combined_graph.add_edges_from(graphx1.edges())

# Add attributes from graph1
for item in graph1:
    if 'source_ref' in item:  # It's an edge
        combined_graph.add_edge(item['source_ref'], item['target_ref'], **item)
    else:  # It's a node
        combined_graph.nodes[item['id']].update(item)

# Function to flatten attributes into a string
def flatten_attributes(data):
    # Remove 'id' from the attributes as it's already used
    data_copy = data.copy()
    data_copy.pop('id', None)
    
    # Convert all values to strings and join them
    return '\n'.join(f"{k}: {v}" for k, v in data_copy.items())

# Create a yFiles GraphWidget
widget = GraphWidget()

# Set nodes in the widget with flattened attributes as labels
widget.set_nodes([
    {
        'id': node,
        'label': flatten_attributes(data),
        'properties': {
            'width': 300, 
            'height': 200,
            'label_visible': True,
            'label_text_size': 12
        }
    }
    for node, data in combined_graph.nodes(data=True)
])

# Set edges in the widget
widget.set_edges([
    {
        'id': i, 
        'start': u, 
        'end': v, 
        'label': f"Edge {u}-{v}",
        'properties': {
            'label_visible': True,
            'label_text_size': 10
        }
    }
    for i, (u, v, data) in enumerate(combined_graph.edges(data=True))
])

# Custom node color mapping function
def node_color_mapping(node):
    return '#15afac'  # You can customize this based on node attributes

# Set the node color mapping
widget.set_node_color_mapping(node_color_mapping)

# Custom node label mapping function
def custom_node_label_mapping(node):
    return node['label']

# Set the node label mapping
widget.set_node_label_mapping(custom_node_label_mapping)

# Apply an organic layoutg
widget.organic_layout()

# Show the widget
widget.show()

# Print node and edge information for verification
print("Nodes:")
for node, data in combined_graph.nodes(data=True):
    print(f"Node {node}: {data}")

print("\nEdges:")
for u, v, data in combined_graph.edges(data=True):
    print(f"Edge {u} - {v}: {data}")

# Debug print to check node labels
print("\nNode Labels:")
for node in widget.nodes:
    print(f"Node {node['id']}: {node['label']}")